In [9]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np
import re

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']
DATA_PATH = os.path.join(LOCAL_PATH, "intermediate_data/cpc")

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

import data_tools as dt
import writing_tools as wt

rng = np.random.default_rng(12898)

RESULTS = {}

MANIFEST = pd.read_csv(os.path.join(DATA_PATH, 'meetings-manifest.csv'))


In [10]:
df = dt.get_analysis_data()
dfa = dt.get_agenda_items()

No data found for 2021-01-14
No data found for 2022-03-17
No data found for 2022-10-13


In [11]:
# Basic info

NUMBER_OF_MEETINGS = df['date'].nunique()
FIRST_MEETING_DATE = pd.to_datetime(df['date']).min()
LAST_MEETING_DATE = pd.to_datetime(df['date']).max()
NUMBER_OF_YEARS = LAST_MEETING_DATE.year - FIRST_MEETING_DATE.year + 1
NUMBER_OF_CASES = len(df)

RESULTS['NumberOfMeetings'] = f"{NUMBER_OF_MEETINGS}"
RESULTS['FirstMeetingDate'] = FIRST_MEETING_DATE.strftime('%Y-%m-%d')
RESULTS['LastMeetingDate'] = LAST_MEETING_DATE.strftime('%Y-%m-%d')
RESULTS['NumberOfYears'] = f"{NUMBER_OF_YEARS}"
RESULTS['NumberOfCases'] = f"{NUMBER_OF_CASES:,.0f}"

In [12]:
# Number of agenda items (total, including non-cases)

AGENDA_ITEMS = 0
for date in df['date'].unique():
    filename = os.path.join(DATA_PATH, date[0:4], date, 'agenda-items.pkl')
    mydf = pd.read_pickle(filename)
    AGENDA_ITEMS += len(mydf)

RESULTS['NumberOfAgendaItems'] = f"{AGENDA_ITEMS:,.0f}"

In [13]:
# Number of supplemental documents

SUPPLEMENTAL_DOCS = 0
for date in df['date'].unique():
    filename = os.path.join(DATA_PATH, date[0:4], date, 'supplemental-docs.pkl')
    mydf = pd.read_pickle(filename)
    SUPPLEMENTAL_DOCS += len(mydf)

RESULTS['NumberOfSupplementalDocs'] = f"{SUPPLEMENTAL_DOCS:,.0f}"

In [14]:
# Page counts

PAGE_COUNT = 0
for date in df['date'].unique():
    row = MANIFEST.loc[MANIFEST['date'] == date].iloc[0]
    PAGE_COUNT += row['agenda_pages'] + row['minutes_pages'] + row['supdocs_pages']

RESULTS['PageCount'] = f"{PAGE_COUNT:,.0f}"
    

In [15]:
# Main summary statistic table

df['unanimity'] = ''
df.loc[ df['n_nays']==0, 'unanimity'] = 'Unanimous'
df.loc[ df['n_nays']==1, 'unanimity'] = '1 Nay'
df.loc[ df['n_nays']>1, 'unanimity'] = '>1 Nays'

summ_tbl = pd.crosstab(df['project_result'], df['unanimity'])

HEADER = """\\begin{tabular}{|lrrrrr|} \\toprule
 & \\multicolumn{3}{c}{Unanimity} & Total & \\\\ 
 & Unanimous & 1 Nay & >1 Nays & & \\\\ \\midrule
\\textit{Project Implication} & & & & & \\\\ """
FOOTER = """\\end{tabular}"""
tbl = []
for idx, row in summ_tbl.iterrows():
    unanimous = f"{row['Unanimous']:,.0f}"
    one_nay = f"{row['1 Nay']:,.0f}"
    multiple_nays = f"{row['>1 Nays']:,.0f}"
    rowsum = f"{row.sum():,.0f}"
    pct = f"({row.sum()/summ_tbl.sum().sum()*100:.1f}\\%)"
    tbl.append([f"~ ~ {idx}", unanimous, one_nay, multiple_nays, rowsum, pct])
unanimous = f"{summ_tbl['Unanimous'].sum():,.0f}"
one_nay = f"{summ_tbl['1 Nay'].sum():,.0f}"
multiple_nays = f"{summ_tbl['>1 Nays'].sum():,.0f}"
total = f"{summ_tbl.sum().sum():,.0f}"
tbl.append([f"~ ~ TOTAL", unanimous, one_nay, multiple_nays, total, ""])
unanimous_pct = f"({summ_tbl['Unanimous'].sum()/summ_tbl.sum().sum()*100:.1f}\\%)"
one_nay_pct = f"({summ_tbl['1 Nay'].sum()/summ_tbl.sum().sum()*100:.1f}\\%)"
multiple_nays_pct = f"({summ_tbl['>1 Nays'].sum()/summ_tbl.sum().sum()*100:.1f}\\%)"
tbl.append(["", unanimous_pct, one_nay_pct, multiple_nays_pct, "", ""])
my_table = wt.latex_table(tbl, header=HEADER, footer=FOOTER)

filename = os.path.join(LOCAL_PATH, 'tables', 'tab_result_unanimity.tex')
with open(filename, 'w') as f:
    f.write(my_table)

print(my_table)

\begin{tabular}{|lrrrrr|} \toprule
 & \multicolumn{3}{c}{Unanimity} & Total & \\ 
 & Unanimous & 1 Nay & >1 Nays & & \\ \midrule
\textit{Project Implication} & & & & & \\ 
~ ~ APPROVED & 365 & 23 & 4 & 392 & (53.9\%) \\ [1ex] 
~ ~ APPROVED IN PART OR WITH MODIFICATIONS & 183 & 17 & 16 & 216 & (29.7\%) \\ [1ex] 
~ ~ DELIBERATIONS CONTINUED TO FUTURE DATE & 108 & 4 & 0 & 112 & (15.4\%) \\ [1ex] 
~ ~ DENIED & 5 & 0 & 2 & 7 & (1.0\%) \\ [1ex] 
~ ~ TOTAL & 661 & 44 & 22 & 727 &  \\ [1ex] 
 & (90.9\%) & (6.1\%) & (3.0\%) &  &  \\ [1ex] 
\end{tabular}



In [16]:
wt.update_results(RESULTS)

{'NumberOfMeetings': '153',
 'FirstMeetingDate': '2018-05-10',
 'LastMeetingDate': '2024-12-19',
 'NumberOfCases': '727',
 'PageCount': '23,633',
 'NumberOfYears': '7',
 'NumberOfAgendaItems': '1,497',
 'NumberOfSupplementalDocs': '6,447'}